# License.
Copyright 2021 Tristan Behrens.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Sampling using the trained model.

In [1]:
import tensorflow as tf
from transformers import GPT2LMHeadModel, TFGPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer
import os
import numpy as np
from source.helpers.samplinghelpers import *
from transformers import GPT2Tokenizer

# Where the checkpoint lives.
# Note can be downloaded from: https://ai-guru.s3.eu-central-1.amazonaws.com/mmm-jsb/mmm_jsb_checkpoints.zip
check_point_path = "" # x LUCIA: ho risistemato i percorsi

# Load the validation data.
validation_data_path = os.path.join(check_point_path, "datasets", "jsb_mmmtrack", "token_sequences_valid.txt")

# Load the tokenizer.
tokenizer_path = 'datasets/jsb_mmmtrack/tokenizer.json' # x LUCIA: messo a mano
tokenizer = Tokenizer.from_file(tokenizer_path)
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_path)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Load the model.
model_path = os.path.join("training", "jsb_mmmtrack", "best_model")
model = GPT2LMHeadModel.from_pretrained(model_path)

print("Model loaded.")

2024-09-20 14:26:29.502438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 14:26:29.524576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 14:26:29.531267: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 14:26:29.553511: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 14:26:30.563517: W tensorflow/comp

Model loaded.


In [2]:
priming_sample, priming_sample_original = get_priming_token_sequence(
    validation_data_path,
    stop_on_track_end=0, # GIOCA CON STOP ON TRACK END OVVERO GUARDA L IMPLEMENTAZIONE... debugga con print_token_sequence(generated_sample)
    stop_after_n_tokens=200,
    return_original=True
)

generated_sample = generate(model, tokenizer, priming_sample)


## QUESTI NON FUNZIONANO PERCHE NON ABBIAMO FLUIDSYNTH
print("Original sample")
#render_token_sequence(priming_sample_original, use_program=False)

print("Reduced sample")
#render_token_sequence(priming_sample, use_program=False)

print("Reconstructed sample")
#render_token_sequence(generated_sample, use_program=False)

Original sample
Reduced sample
Reconstructed sample


In [3]:
print_token_sequence(generated_sample)



0000 PIECE_START
0001   TRACK_START
0002     INST=2
0003     DENSITY=2
0004     BAR_START
0005       NOTE_ON=64
0006       TIME_DELTA=4.0
0007       NOTE_OFF=64
0008     BAR_END
0009     BAR_START
0010       NOTE_ON=66
0011       TIME_DELTA=2.0
0012       NOTE_OFF=66
0013       NOTE_ON=64
0014       TIME_DELTA=2.0
0015       NOTE_OFF=64
0016       NOTE_ON=62
0017       TIME_DELTA=4.0
0018       NOTE_OFF=62
0019       NOTE_ON=64
0020       TIME_DELTA=4.0
0021       NOTE_OFF=64
0022       NOTE_ON=57
0023       TIME_DELTA=4.0
0024       NOTE_OFF=57
0025     BAR_END
0026     BAR_START
0027       NOTE_ON=57
0028       TIME_DELTA=4.0
0029       NOTE_OFF=57
0030       NOTE_ON=56
0031       TIME_DELTA=4.0
0032       NOTE_OFF=56
0033       NOTE_ON=57
0034       TIME_DELTA=4.0
0035       NOTE_OFF=57
0036       NOTE_ON=64
0037       TIME_DELTA=4.0
0038       NOTE_OFF=64
0039     BAR_END
0040     BAR_START
0041       NOTE_ON=64
0042       TIME_DELTA=4.0
0043       NOTE_OFF=64
0044       NOTE_ON=64

In [4]:
print_token_sequence(priming_sample)

0000 PIECE_START
0001   TRACK_START
0002     INST=2
0003     DENSITY=2
0004     BAR_START
0005       NOTE_ON=64
0006       TIME_DELTA=4.0
0007       NOTE_OFF=64
0008     BAR_END
0009     BAR_START
0010       NOTE_ON=66
0011       TIME_DELTA=2.0
0012       NOTE_OFF=66
0013       NOTE_ON=64
0014       TIME_DELTA=2.0
0015       NOTE_OFF=64
0016       NOTE_ON=62
0017       TIME_DELTA=4.0
0018       NOTE_OFF=62
0019       NOTE_ON=64
0020       TIME_DELTA=4.0
0021       NOTE_OFF=64
0022       NOTE_ON=57
0023       TIME_DELTA=4.0
0024       NOTE_OFF=57
0025     BAR_END
0026     BAR_START
0027       NOTE_ON=57
0028       TIME_DELTA=4.0
0029       NOTE_OFF=57
0030       NOTE_ON=56
0031       TIME_DELTA=4.0
0032       NOTE_OFF=56
0033       NOTE_ON=57
0034       TIME_DELTA=4.0
0035       NOTE_OFF=57
0036       NOTE_ON=64
0037       TIME_DELTA=4.0
0038       NOTE_OFF=64
0039     BAR_END
0040     BAR_START
0041       NOTE_ON=64
0042       TIME_DELTA=4.0
0043       NOTE_OFF=64
0044       NOTE_ON=64

In [5]:
from note_seq.midi_io import note_sequence_to_midi_file # GOOGLATO -> https://wlt.coffee/cream/docs/note_seq/midi_io.html

priming_ns = token_sequence_to_note_sequence(priming_sample) # GUARDA IN samplinghelpers.py

note_sequence_to_midi_file(priming_ns, 'priming.mid')


In [6]:
priming_original_ns = token_sequence_to_note_sequence(priming_sample_original) 
note_sequence_to_midi_file(priming_ns, 'priming_original.mid')

In [7]:
generated_sample_ns = token_sequence_to_note_sequence(generated_sample) 

note_sequence_to_midi_file(generated_sample_ns, 'Sperimentazione4.mid')